<a href="https://colab.research.google.com/github/helidenial/drt/blob/main/torre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install python3-pip >/etc/null 2>&1 ; echo "|"
!python -m pip install --upgrade pip setuptools wheel >/etc/null 2>&1 ; echo "||"
!python -m pip install lbry-libtorrent >/etc/null 2>&1 ; echo "|||"
!apt install python3-libtorrent >/etc/null 2>&1 ; echo "||||"

import time
from IPython.display import display
import ipywidgets as widgets
import libtorrent as lt
import requests
from google.colab import drive
import requests as re
from bs4 import BeautifulSoup as Soup

drive.mount("/content/drive")

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

def sendMessage(bot_message):
    token = ''
    chatID = ''
    send_text = 'https://api.telegram.org/bot' + token + '/sendMessage?chat_id=' + chatID + '&parse_mode=Markdown&text=' + bot_message
    res = requests.get(send_text)
    return res

def getYifyFor(movie_name):
  base_link = "https://yts.mx/movies/"
  movie = movie_name.replace(' ','-').lower()
  res = re.get(base_link+movie)

  soup = Soup(res.content)

  link_a = soup.find('div',{"class":"bottom-info"}).findAll('a')
  for i,a in enumerate(link_a):
    try:
      print(f'[{i}]',a['title'],a['href'])
    except:
      pass

  inp = input("Enter which link to download")
  
  return link_a[int(inp)]['href']

def downloadFromYify(file_link):
  !wget $file_link
  file_name = file_link.split('/')[-1]
  params = {
    "save_path": "/content/drive/Shareddrives/tdHackgence/Torrent",
    "ti": lt.torrent_info(file_name),
  }
  downloads.append(ses.add_torrent(params))
 
location=""

def show_bars():
  name =''
  state_str = [
      "queued",
      "checking",
      "downloading metadata",
      "downloading",
      "finished",
      "seeding",
      "allocating",
      "checking fastresume",
  ]
  
  layout = widgets.Layout(width="auto")
  style = {"description_width": "initial"}
  download_bars = [
      widgets.FloatSlider(
          step=0.01, disabled=True, layout=layout, style=style
      )
      for _ in downloads
  ]
  display(*download_bars)
  
  while downloads:
      next_shift = 0
      for index, download in enumerate(downloads[:]):
          bar = download_bars[index + next_shift]
          if not download.is_seed():
              s = download.status()
  
              bar.description = " ".join(
                  [
                      download.name(),
                      str(s.download_rate / 1000),
                      "kB/s",
                      state_str[s.state],
                  ]
              )
              bar.value = s.progress * 100
          else:
              next_shift -= 1
              ses.remove_torrent(download)
              downloads.remove(download)
              bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
              download_bars.remove(bar)
              name = download.name()
              print(download.name(), "complete")
      time.sleep(1)


  sendMessage('Uploaded \n\n'+name+'\n\nTo ' + location )
  print('Done uploading '+name+' To your google drive')


import os
home_path = "/content/drive/Shareddrives/tdHackgence/Torrent/Subtitles"

def getSubsFor(sub_link):  
  temp_path = os.getcwd()
  os.chdir(home_path)
  res = re.get(sub_link)
  sub_link
  sou = Soup(res.content)
  movie="train-to-busan"
  res = sou.find("span",{"class":"flag flag-gb"})
  a_href = res.parent.parent.find('a')

  final_link="https://yifysubtitles.org"+a_href['href'].replace('subtitles','subtitle')+".zip"
  file_name = final_link.split('/')[-1]
  ! wget $final_link
  ! unzip $file_name
  os.chdir(temp_path)

In [ ]:
params = {"save_path": "/content/drive/Shareddrives/tdHackgence/Torrent"}
 
while True:
    magnet_link =''
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )
 
location = "Shareddrives/tdHackgence/Torrent"

In [ ]:

'''

var id = setInterval(()=>{console.log("working");document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();},3*60*1000);

var btn = document.querySelector("#ok") 
if (btn != null ) document.querySelector("#ok").click()

'''

In [ ]:
params = {"save_path": "/content/drive/Shareddrives/tdHackgence/Torrent/AOT-Manga"}
 
while True:
    magnet_link =''
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

location = "Shareddrives/tdHackgence/Torrent"

In [ ]:
downloadFromYify(getYifyFor(input("Enter the movie name with year of release:")))

In [ ]:
show_bars()
downloads=[]

In [ ]:
downloads=[]

In [ ]:
 ''' 
  All the functions so far : 
    
   sendMessage(msg) -> send msg to telegram. 
   getYifyFor(movie_name) -> get_links 
   downloadFromYify(file_link) -> Download from yify  
   show_bars() -> Shows download bars 
    
   '''


In [ ]:
getSubsFor(getYifyFor(input("Enter movie name with year of release ")))